In [1]:
import synapseclient
syn = synapseclient.Synapse()
syn.login()

[WARNING] Logging into Synapse via apikeys will be deprecated by early 2024.  If you've used `rememberMe=True` in the past, you may also be getting this message, please delete your ~/.synapseCache/.session file. This message will disappear if you use a Synapse Personal Access Token to login.


Welcome, Jineta Banerjee!



In [2]:
from synapseclient import Activity
from synapseclient import Entity, Project, Folder, File, Link
from synapseclient import Evaluation, Submission, SubmissionStatus
from synapseclient import Wiki
import synapseutils
from re import search
import pandas as pd

## Make dataset with all cram and crai files

syn52357115 contains all aligned files for all samples.

In [3]:
# Make a dataset by recursively adding folders:
dataset = syn.get("syn52357115") #pull in empty dataset entity

# Add a list of Folders, overwriting any existing files in the dataset
dataset.add_folders(["syn51832064", 
"syn51833559",
"syn51833711",
"syn51834988",
"syn51835488",
"syn51835546",
"syn52601592", #jh-batch1-missed-samples
"syn52593527", #mixed-kit-samples
"syn52654065", #wu-batches-corrected-samples
"syn52593659" #wu-batches-missed-samples
], force=True)

dataset = syn.store(dataset)

## Make dataset with strelka and mutect2 vcf files

In [13]:
# Make a dataset by recursively adding folders:
dataset = syn.get("syn52656598") #pull in empty dataset entity

# Add a list of Folders, overwriting any existing files in the dataset
dataset.add_folders(["syn52601661", #jh-batch1-missed-samples 
"syn51832454", #jh-batch1
"syn52593603", #mixed-kit-samples
"syn52654119", #wu-batches-corrected-samples
"syn52593771", #wu-batches-missed-samples
"syn51833586", #wu-batch1-batch2
"syn51834214", #wubatch1-new
"syn51835402" #wu-batch2-new
#"syn51835521", #wu-pdx-batch1-new
#"syn51835594" #wu-pdx-batch2-new
], force=True)

dataset = syn.store(dataset)

select only vcf.gz and vcf.gz.tbi files

In [14]:
dataset = syn.get("syn52656598")
df2 = pd.DataFrame(columns=["files", "synid"])
for f in dataset.datasetItems:
   f_id = f.get('entityId')
   filename = syn.get(f_id, downloadFile=False).get('name')
   df = pd.DataFrame(filename.split("\n"), columns = ['files'])
   df['synid'] = f_id
   df2 = df2.append(df)   

selected_files = df2[df2["files"].str.endswith('mutect2.filtered.vcf.gz') | 
                     df2["files"].str.endswith('mutect2.filtered.vcf.gz.tbi') | 
                     df2["files"].str.endswith('strelka.somatic_indels.vcf.gz') |
                     df2["files"].str.endswith('strelka.somatic_indels.vcf.gz.tbi') |
                     df2["files"].str.endswith('strelka.somatic_snvs.vcf.gz') |
                     df2["files"].str.endswith('strelka.somatic_snvs.vcf.gz.tbi')]

unwanted_files = df2[~df2.synid.isin(selected_files.synid)]

# remove specific files from the dataset
for i in range(len(unwanted_files.synid)):
    #print(i)
    #print(unwanted_files.synid.iloc[i])
    dataset.remove_item(unwanted_files.synid.iloc[i])

dataset = syn.store(dataset)

Alternate method to preselect items to add to dataset (not recommended if the number of items in parent folder is too many)

In [ ]:
files = synapseutils.sync.syncFromSynapse(syn, "syn51831408", downloadFile=False)
files_df = pd.DataFrame(files)
files_df = files_df[["id", 'name', 'versionNumber']]

selected_files = files_df[files_df["name"].str.endswith('mutect2.filtered.vcf.gz') | 
                     files_df["name"].str.endswith('mutect2.filtered.vcf.gz.tbi') | 
                     files_df["name"].str.endswith('strelka.somatic_indels.vcf.gz') |
                     files_df["name"].str.endswith('strelka.somatic_indels.vcf.gz.tbi') |
                     files_df["name"].str.endswith('strelka.somatic_snvs.vcf.gz') |
                     files_df["name"].str.endswith('strelka.somatic_snvs.vcf.gz.tbi')]

items = selected_files[["id", "versionNumber"]]
items.columns = ["entityId", "versionNumber"]
items = items.to_dict('records')

dataset = synapseclient.Dataset(
    name="NF-OSI-processed-WES-strelka-mutect2-vcfs-2023",
    parent="syn4939902",
    dataset_items = items
)
dataset = syn.store(dataset)

## Download items to local machine

In [ ]:
# Downloading items from a dataset:

df = syn.get("syn52656598")
for f in df.datasetItems:
   f_id = f.get('entityId')
   filename = syn.get(f_id, downloadFile=False, downloadLocation = "add path here")
   print(f_id)